So far, this program has covered decision trees growing with an entropy criterion. In doing so, however, the program glossed over how that actually works. In this checkpoint, you'll get more detail into how an algorithm that does that would practically function.

Here, you'll learn about one popular algorithm for building a decision tree using entropy and information gain: *ID3*.

## The ID3 algorithm

ID3 stands for *Iterative Dichotomizer 3*, and it's one of the simplest ways to create a decision tree. ID3 can be generalized into more robust scenarios, but the implementation is based on the framework that you'll go over here. Essentially, ID3 goes through every feature to find the most valuable attribute, and then it splits the node based on that attribute. Then it moves further and further down the tree until it either has a pure class or meets a terminating condition. The details are explored further below.

Before you can start working with ID3, however, there are some requirements for the data in this most basic form. Firstly, outcomes have to be binary. The simplest version of ID3 is a binary classifier. Also, the attributes that you use to build the tree have to be categorical. Attributes can have many categories, but they must be known and countable.

If those two criteria are met, then you can build a basic ID3 classifying algorithm.

The other thing that you'll need for this is the definition of entropy. Recall from the previous assignment that you're going to use Shannon Entropy $H$, defined as follows:

$$ H = -\sum_{i=1}^n P(x_i)log_2 P(x_i) $$

For simplicity of calculation, you're actually going to do a slight transform on this definition. Recall from a (quite possibly long ago) algebra class that you can bring exponentials out to the front of a logarithm. In this case, you'll raise the probability to `-1`, changing the formula to the following:

$$ H = \sum_{i=1}^n P(x_i)log_2 \frac{1}{P(x_i)} $$

This removes the negative sign up front, and will make it easier for you to implement this formula.

## Calculating entropy

Because this algorithm is based on entropy, go over a quick example of how to calculate it.

Say that you have 20 students, and you're trying to classify them as male or female. The only attribute that you have is whether their height is tall, medium, or short. Of the 20 students, 12 are boys and 8 are girls. Of the 12 boys, 4 are tall, 6 are medium, and 2 are short. Of the 8 girls, 1 is tall, 2 are medium, and 5 are short.

What is the entropy before any rule is applied? And what is the entropy after applying a rule for being tall?

The initial entropy is just the formula plugged in over both the possible classes of interest:

$$ H = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$


$$ = \frac{12}{20}*log_2 \frac{20}{12} + \frac{8}{20}*log_2 \frac{20}{8} = .971 $$

What if you apply the rule `height = short`? You need to calculate the weighted average of the two entropies, one for the short students and one for the students who aren't short.

$$ H(short) = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$

$$ = \frac{2}{7}*log_2 \frac{7}{2} + \frac{5}{7}*log_2 \frac{7}{5} = .863 $$

$$ H(not\_short) = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$

$$ = \frac{10}{13}*log_2 \frac{13}{10} + \frac{3}{13}*log_2 \frac{13}{3} = .779 $$

Note that all the probabilities here are conditional on the criteria that you're assuming (short or not short). The weighted average of the two is this:

$$ P(short) * H(short) + P(not\_short) * H(not\_short) = \frac{7}{20} * .863 + \frac{13}{20} * .779 = .809 $$

So the entropy from this question would go from `0.971` to `0.809`. That's an improvement! Use the space below to calculate the entropy of the other criteria, because you also know whether the students are tall or medium.

In [1]:
from math import log2

# Put your calculations below















# The example solution is below here. Don't peek!





In [2]:
# Example solution

# Tall
H_tall = 4 / 5 * log2(5 / 4) + 1 / 5 * log2(5 / 1)
H_not_tall = 8 / 15 * log2(15 / 8) + 7 / 15 * log2(15 / 7)

entropy_tall = 5 / 20 * H_tall + 15 / 20 * H_not_tall


# Medium
H_medium = 6/8 * log2(8/6) + 2/8 * log2(8/2)
H_not_medium = 6/12 * log2(12/6) + 6/12 * log2(12/6)

entropy_medium = 8/20 * (H_medium) + 12/20 * (H_not_medium)

print(entropy_tall, entropy_medium)

0.9280757477080679 0.9245112497836532


You should have found entropies of `0.925` for medium and `0.928` for tall. Both of those entropies are higher. As you know, you want to prioritize the questions with the most information gain. Which one of these questions would that be?

Asking if an individual is short provides the most information gain. Also, note that for all possible questions, you're still comparing with the same initial entropy value. So one way of seeing which question has the most information gain is to find the one with the lowest entropy. This makes sense when you think of entropy as uncertainty; the less uncertainty after a question, the more information you gained.

## Pseudocoding the algorithm

*Pseudocode* is the process of writing the steps and logic that you would implement in code, but in normal language rather than in commands that a programming language could execute. It can be a useful way to chart out how you want to build an algorithm, and it's a common topic for technical interviews. Here you'll use pseudocode to explore the ID3 algorithm.

Here is some reasonable pseudocode for ID3. This pseudocode will then be followed up with an explanation of the steps. The outcome for this variable will be `A` or `B`. An attribute is denoted as a<sub>i</sub>, and a value of that attribute is v<sub>i</sub>.


<pre>
Algorithm(Observations, Outcome, Attributes)
    Create a root node.
    If all observations are 'A', label root node 'A' and return.
    If all observations are 'B', label root node 'B' and return.
    If no attributes return the root note labeled with the most common Outcome.
    Otherwise, start:
        For each value v<sub>i</sub> of each attribute a<sub>i</sub>, calculate the entropy.
        The attribute a<sub>i</sub> and value v<sub>i</sub> with the lowest entropy is the best rule.
        The attribute for this node is then a<sub>i</sub>
            Split the tree to below based on the rule a<sub>i</sub> = v<sub>i</sub>
            Observations<sub>v<sub>i</sub></sub> is the subset of observations with value v<sub>i</sub>
            If Observations<sub>v<sub>i</sub></sub> is empty cap with node labeled with most common Outcome
            Else at the new node start a subtree (Observations<sub>v<sub>i</sub></sub>, Target Outcome, Attributes - {a<sub>i</sub>}) and repeat the algorithm
</pre>

Now, walk through this pseudocode algorithm in plain English, piece by piece.

First, you create a root node. Simple enough—you have to start with something.

The next two lines say that if you're already exclusively one class, just label with that class and you're done. Similarly, the following line says that if you don't have any attributes left, you're also done, labeling with the most common outcome.

Then you get into the real algorithm. First, you have to find the best attribute by calculating entropies for all possible values. The attribute-value pair with the lowest entropy is then the best attribute. That's the attribute that you give to the node.

You use that rule to split the node, creating subsets of observations. There are then two new nodes, each with a subset of the observations corresponding to the rule. If observations are null, then label with the dominant outcome.

Otherwise, at each new node, start the algorithm again.

This is how a decision tree actually functions. Understanding this will give you some insight into how this algorithm works and reveals several attributes of the algorithm. Firstly, the solution is not necessarily optimal. The tree can get stuck in local optima, just like with the gradient descent algorithm. It also has no way to work backward if it finds itself in an informationless space. You can add criteria to make it stop before the tree has grown to run out of attributes or all leaf nodes are single classes.

## Drill

Look over the code for [this real ID3 algorithm in Python](https://github.com/NinjaSteph/DecisionTree/blob/master/sna2111_DecisionTree/DecisionTree.py). Note how well the author breaks up functionality into individual, reusable, and well-named helper functions. See if you can match the pseudocode steps to the code in this example.